In [1]:
# script to calculate ratios for use in TTS method 
# match rf 06 to varying BL for ratios 
# 1) RF06 BL before anthropogenic from 
# 2) RF06 BL after anthropogenic from 
# author: sofia chelpon
# date created: 2-17-2020

In [2]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns
import datetime as dt
from scipy.optimize import least_squares

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [3]:
# load in both toga and awas data pickles 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_replace_twp_noRF06.pkl")
awas_df_bl = awas_df[awas_df['GGALT'] < 2000]
awas_df_bl_all = awas_df_bl 
#awas_df_bl_north = awas_df_bl[awas_df_bl['GGLAT'] > 10]
awas_df_bl_south = awas_df_bl[awas_df_bl['GGLAT'] < 10]

toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df_twp_noRF06.pkl")
#toga_df = toga_df.drop(columns = 'WIC')
toga_df_bl = toga_df[toga_df['GGALT'] < 2000]
toga_df_bl_all = toga_df_bl 
#toga_df_bl_north = toga_df_bl[toga_df_bl['GGLAT'] > 10]
toga_df_bl_south = toga_df_bl[toga_df_bl['GGLAT'] < 10]

In [4]:
toga_df

,Time_UTC,Flight,Notes,Intrument,GGALT,GGLAT,GGLON,Acetaldehyde,n_Butane,i_Butane,Benzene,C3H8,Methanol,Bromoform,Tetrachloroethylene,Dibromomethane,Dichloromethane,Chloroform,Carbon_Tetrachloride
0,2014-01-17 00:18:06,RF03,"Domain survey, 1 - Guam to SE (Chuuk)",TOGA,3127.166016,13.379817,144.993500,191.199997,1.51,2.49,NaN,41.0,611.0,0.689,0.15,NaN,44.490002,9.39,97.099998
1,2014-01-17 00:20:06,RF03,"Domain survey, 1 - Guam to SE (Chuuk)",TOGA,4212.603027,13.258845,145.136200,126.800003,1.68,2.60,NaN,13.0,378.0,0.480,0.15,NaN,50.110001,10.29,94.300003
2,2014-01-17 00:22:06,RF03,"Domain survey, 1 - Guam to SE (Chuuk)",TOGA,5284.313477,13.121677,145.297684,NaN,1.42,2.17,7.280000,18.0,228.0,0.050,0.15,NaN,42.029999,8.67,96.699997
3,2014-01-17 00:24:06,RF03,"Domain survey, 1 - Guam to SE (Chuuk)",TOGA,6472.384277,12.977636,145.466934,116.900002,1.14,1.09,8.230000,5.0,327.0,0.625,0.15,1.043,43.689999,9.42,94.599998
4,2014-01-17 00:26:06,RF03,"Domain survey, 1 - Guam to SE (Chuuk)",TOGA,7449.240723,12.823807,145.647217,NaN,3.01,2.26,5.280000,22.0,10.0,0.050,0.15,1.001,49.049999,9.29,93.400002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2115,2014-02-25 07:10:04,RF15,"Jet crossing and LS survey, 4",TOGA,3573.252930,13.389064,144.071243,115.300003,2.41,2.04,10.150000,48.0,738.0,0.525,0.89,0.733,45.919998,9.18,79.400002
2116,2014-02-25 07:12:04,RF15,"Jet crossing and LS survey, 4",TOGA,3202.747559,13.389287,144.240921,103.199997,5.10,4.83,24.260000,63.0,905.0,0.575,1.01,0.666,67.750000,11.60,79.800003
2117,2014-02-25 07:14:04,RF15,"Jet crossing and LS survey, 4",TOGA,2282.634277,13.389470,144.391220,233.300003,2.79,2.47,14.910000,60.0,931.0,0.644,1.06,0.927,51.220001,10.22,77.099998
2118,2014-02-25 07:16:04,RF15,"Jet crossing and LS survey, 4",TOGA,1030.910278,13.386702,144.540161,323.799988,3.55,3.29,17.389999,75.0,905.0,0.561,1.28,0.890,55.549999,10.87,79.099998


In [5]:
################# -------------------- TOGA  -------------------- #################

In [6]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
toga_means = pd.DataFrame()

toga_means['ut'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)].mean()
toga_means['full_bl'] = toga_df_bl_all.mean()
#toga_means['north_bl'] = toga_df_bl_north.mean()
toga_means['south_bl'] = toga_df_bl_south.mean()

toga_means = toga_means.drop('GGALT').drop('GGLAT').drop('GGLON')
#toga_means.index.name = "Trace_Gas"
toga_means = toga_means.reset_index()
toga_means = toga_means.rename(columns={'index': 'Trace_Gas'})
toga_means

,Trace_Gas,ut,full_bl,south_bl
0,Acetaldehyde,19.476095,190.618576,206.103989
1,n_Butane,2.805677,11.938943,8.400274
2,i_Butane,2.487222,8.592068,6.504587
3,Benzene,8.707263,21.827354,21.655966
4,C3H8,35.808945,75.855766,68.550461
5,Methanol,235.399734,647.898071,671.205627
6,Bromoform,0.612737,0.935894,0.977303
7,Tetrachloroethylene,0.956070,1.494904,1.538899
8,Dibromomethane,1.017580,1.056000,1.075766
9,Dichloromethane,43.360123,56.301704,59.695469


In [7]:
# ratios for 1) flights in total and 2) individual flights 
toga_ratios = pd.DataFrame()

toga_ratios['Trace_Gas'] = toga_means['Trace_Gas']
toga_ratios['full_BL'] = toga_means['ut']/toga_means['full_bl']
#toga_ratios['north_BL'] = toga_means['ut']/toga_means['north_bl']
toga_ratios['south_BL'] = toga_means['ut']/toga_means['south_bl']

toga_ratios

# drop species names so doesnt duplicate column when merged 
toga_ratios = toga_ratios.drop('Trace_Gas', axis = 1)

In [8]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_08042020.xlsx'
toga_lifetimes = pd.read_excel(mypath2)

# merge lifetimes to ratios
toga_ratios_full = toga_lifetimes.merge(toga_ratios, left_index=True, right_index=True)
toga_ratios_full.insert(0, 'Instrument', 'TOGA')
toga_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,full_BL,south_BL
0,TOGA,Acetaldehyde,0.283684,0.321694,0.348720,0.102173,0.094496
1,TOGA,n_Butane,1.979710,3.901968,8.096203,0.235002,0.333998
2,TOGA,i_Butane,2.193483,4.101641,7.916587,0.289479,0.382380
3,TOGA,Benzene,3.819659,6.602596,12.137543,0.398915,0.402072
4,TOGA,C3H8,4.276704,9.379963,22.541651,0.472066,0.522374
5,TOGA,Methanol,5.198347,9.757627,20.088572,0.363328,0.350712
6,TOGA,Bromoform,12.898712,18.666902,24.882113,0.654708,0.626967
7,TOGA,Tetrachloroethylene,28.154334,83.261722,320.000000,0.639553,0.621269
8,TOGA,Dibromomethane,39.940949,100.000000,310.000000,0.963617,0.945912
9,TOGA,Dichloromethane,46.722142,120.000000,350.000000,0.770139,0.726355


In [9]:
################# -------------------- AWAS -------------------- #################

In [10]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
awas_means = pd.DataFrame()

# full bl 
awas_means['ut'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)].mean()
awas_means['full_bl'] = awas_df_bl_all.mean()
#awas_means['north_bl'] = awas_df_bl_north.mean()
awas_means['south_bl'] = awas_df_bl_south.mean()

awas_means = awas_means.drop('GGALT').drop('GGLAT').drop('GGLON')
#toga_means.index.name = "Trace_Gas"
awas_means = awas_means.reset_index()
awas_means = awas_means.rename(columns={'index': 'Trace_Gas'})
awas_means

,Trace_Gas,ut,full_bl,south_bl
0,C2H6_Ethane,482.961548,759.592590,698.719971
1,C2H2_Ethyne,54.076923,105.423080,91.839996
2,C3H8_Propane,37.423077,112.740738,79.160004
3,C4H10_Isobutane,2.506410,10.381481,5.372000
4,C4H10_nButane,4.029487,15.848148,8.056001
5,C5H12_nPentane,0.626882,2.300000,0.952381
6,C6H6_Benzene,9.298717,22.855772,19.955999
7,OCS_Carbonyl_sulfide,548.316101,545.849060,542.958313
8,C2H6S_Dimethyl_sulfide,0.371333,17.039997,12.801667
9,CCl2F2_CFC_12,518.858154,517.442322,516.260864


In [11]:
# ratios for 1) flights in total and 2) individual flights 
awas_ratios = pd.DataFrame()

awas_ratios['Trace_Gas'] = awas_means['Trace_Gas']
awas_ratios['full_BL'] = awas_means['ut']/awas_means['full_bl']
#awas_ratios['north_BL'] = awas_means['ut']/awas_means['north_bl']
awas_ratios['south_BL'] = awas_means['ut']/awas_means['south_bl']

awas_ratios

# drop species names so doesnt duplicate column when merged 
awas_ratios = awas_ratios.drop('Trace_Gas', axis = 1)

In [12]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_08042020.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
len(awas_lifetimes)

# merge lifetimes to ratios 
awas_ratios_full = awas_lifetimes.merge(awas_ratios, left_index=True, right_index=True)
awas_ratios_full.insert(0, 'Instrument', 'AWAS')
awas_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,full_BL,south_BL
0,AWAS,C2H6_Ethane,18.996560,51.867565,170.000000,0.635817,0.691209
1,AWAS,C2H2_Ethyne,5.460657,9.761061,18.790019,0.512951,0.588817
2,AWAS,C3H8_Propane,4.276704,9.379963,22.541651,0.331939,0.472752
3,AWAS,C4H10_Isobutane,2.193483,4.101641,7.916587,0.241431,0.466569
4,AWAS,C4H10_nButane,1.979710,3.901968,8.096203,0.254256,0.500185
5,AWAS,C5H12_nPentane,1.225894,2.360346,4.742530,0.272557,0.658226
6,AWAS,C6H6_Benzene,3.819659,6.602596,12.137543,0.406843,0.465961
7,AWAS,OCS_Carbonyl_sulfide,2000.000000,2000.000000,2000.000000,1.004520,1.009868
8,AWAS,C2H6S_Dimethyl_sulfide,0.655356,0.377066,0.407900,0.021792,0.029007
9,AWAS,CCl2F2_CFC_12,37000.000000,37000.000000,37000.000000,1.002736,1.005031


In [13]:
################# -------------------- MERGE THE TWO  -------------------- #################

In [14]:
master_list = awas_ratios_full
master_list = master_list.append(toga_ratios_full, ignore_index = True, sort = None)

# pickle this!! 
master_list.to_pickle("./contrast_ratios_varybl_awas_replace_noRF06.pkl")

# look at output 
master_list

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,full_BL,south_BL
0,AWAS,C2H6_Ethane,18.996560,51.867565,170.000000,0.635817,0.691209
1,AWAS,C2H2_Ethyne,5.460657,9.761061,18.790019,0.512951,0.588817
2,AWAS,C3H8_Propane,4.276704,9.379963,22.541651,0.331939,0.472752
3,AWAS,C4H10_Isobutane,2.193483,4.101641,7.916587,0.241431,0.466569
4,AWAS,C4H10_nButane,1.979710,3.901968,8.096203,0.254256,0.500185
5,AWAS,C5H12_nPentane,1.225894,2.360346,4.742530,0.272557,0.658226
6,AWAS,C6H6_Benzene,3.819659,6.602596,12.137543,0.406843,0.465961
7,AWAS,OCS_Carbonyl_sulfide,2000.000000,2000.000000,2000.000000,1.004520,1.009868
8,AWAS,C2H6S_Dimethyl_sulfide,0.655356,0.377066,0.407900,0.021792,0.029007
9,AWAS,CCl2F2_CFC_12,37000.000000,37000.000000,37000.000000,1.002736,1.005031
